In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import fiona
from shapely.geometry import Point, shape
from gisutils import df2shp, project
from pydrograph import Nwis
import geopandas as gp

# Notebook to extract NWIS data from model area and make obs tables

## 1. Create model bounding box and retrieve data using the excellent `pydrogrpah`

In [ ]:
extent_shp = '../processed_input/extend_15km_buffer.shp'
epsg = 5070

In [ ]:
extent_poly = shape(fiona.open(extent_shp).next()['geometry'])
extent_poly_ll = project(extent_poly, "+init=epsg:{}".format(epsg), "+init=epsg:4269")
extent_poly_ll.bounds

In [ ]:
nwis = Nwis(extent=extent_poly_ll)

## 2. Get all daily values

In [ ]:
all_dvs = nwis.get_siteinfo('daily_values')

In [ ]:
all_dvs.index = ['{:08d}'.format(i) for i in all_dvs.index]
all_dvs

### 2.1 Export daily values to shapefile

In [ ]:
df2shp(all_dvs,'../obs_data/NWIS_DV_sites.shp', epsg=4269)

### 2.2 Create and export daily values obs data .csv 

In [ ]:
all_dvs_gdf = gp.GeoDataFrame(all_dvs, crs="EPSG:4269", geometry=all_dvs.geometry)

**Plot with buffer to make sure geometry was defined correctly**

In [ ]:
x,y = extent_poly_ll.exterior.xy
fig, ax = plt.subplots()
plt.plot(x,y)
all_dvs_gdf.plot(ax=ax, c='red')
plt.show()

**Reproject gdf to match model CRS (EPSG 5070); add x, y, and obsname columns for `mfsetup` obs block,**

In [ ]:
all_dvs_gdf_reproj = all_dvs_gdf.to_crs(epsg=epsg)
all_dvs_gdf_reproj['x'] = all_dvs_gdf_reproj.geometry.x
all_dvs_gdf_reproj['y'] = all_dvs_gdf_reproj.geometry.y
all_dvs_gdf_reproj['obsprefix'] = all_dvs_gdf_reproj.index

In [ ]:
all_dvs_gdf_reproj

In [ ]:
all_dvs_gdf_reproj.to_csv('../obs_data/NWIS_DV_sites.csv')

## 3. Get groundwater daily values

In [ ]:
gw_dv = nwis.get_siteinfo('gwdv')
gw_dv

In [ ]:
#  pydrograh gw dv data retrieval not working at the momnet. do it manually for now -- see below:

# gw_data = nwis.get_all_dvs([gw_site], 
#                            parameter_code='72019',
#                            start_date='2009-01-01',
#                            end_date='2016-01-01'
#                           )

# gw_data

## Data pulled mannualy from the site in the model ` 414421074222801` using url:

https://waterdata.usgs.gov/nwis/dv?cb_72019=on&format=rdb&site_no=414525074360601&referred_module=sw&period=&begin_date=2009-01-01&end_date=2016-01-01

saved data at '../obs_data/nwis_gw_dv_raw.txt'

In [ ]:
gw_data = pd.read_csv('../obs_data/nwis_gw_dv_raw.txt', skiprows=17, delim_whitespace=True, comment='#')
gw_data = gw_data.groupby('site_no').mean()
gw_data = gw_data.rename(columns={'106190_72019_00003':'ave_dtw_ft'})
gw_data = gw_data.join(gw_dv[['alt_va']])
gw_data['gw_elev_ft'] = gw_data['alt_va'] - gw_data['ave_dtw_ft']
gw_data['gw_elev_m'] = gw_data['gw_elev_ft'] * 0.3048
gw_data.to_csv('../obs_data/NWIS_GW_DV_data.csv')
gw_data

### 3.1 Export groundwater daily values to shapefile

In [ ]:
df2shp(gw_dv,'../obs_data/NWIS_GW_DV_sites.shp', epsg=4269)

### 3.2 Create and export groundwater daily values obs data .csv 

In [ ]:
gw_dv_gdf = gp.GeoDataFrame(gw_dv, crs="EPSG:4269", geometry=gw_dv.geometry)

fig, ax = plt.subplots()
plt.plot(x,y)
gw_dv_gdf.plot(ax=ax, c='red')
plt.show()

In [ ]:
gw_dv_gdf_reproj = gw_dv_gdf.to_crs(epsg=epsg)
gw_dv_gdf_reproj['x'] = gw_dv_gdf_reproj.geometry.x
gw_dv_gdf_reproj['y'] = gw_dv_gdf_reproj.geometry.y
gw_dv_gdf_reproj['obsprefix'] = gw_dv_gdf_reproj.index
gw_dv_gdf_reproj

In [ ]:
gw_dv_gdf_reproj.to_csv('../obs_data/NWIS_GW_DV_sites.csv', index=False)

# 4. Get field measurements

In [ ]:
nwis_fm = nwis.get_siteinfo('field_measurements')
nwis_fm

In [ ]:
#  get rid of DIVERSION FROM NEVERSINK RESERVOIR NY -- 01435800; not a useful streamflow measurement
nwis_fm = nwis_fm.loc[nwis_fm.site_tp_cd == 'ST']

nwis_fm.index = ['{:08d}'.format(i) for i in nwis_fm.index]
nwis_fm.shape

### 4.1 Export field measurements to shapefile

In [ ]:
df2shp(nwis_fm,'../obs_data/NWIS_FieldMeas_sites.shp', epsg=4269)

### 4.2 Create and export field measurements obs data .csv 

In [ ]:
nwis_fm_gdf = gp.GeoDataFrame(nwis_fm, crs="EPSG:4269", geometry=nwis_fm.geometry)

fig, ax = plt.subplots()
plt.plot(x,y)
nwis_fm_gdf.plot(ax=ax, c='red')
plt.show()

In [ ]:
nwis_fm_gdf_reproj = nwis_fm_gdf.to_crs(epsg=epsg)
nwis_fm_gdf_reproj['x'] = nwis_fm_gdf_reproj.geometry.x
nwis_fm_gdf_reproj['y'] = nwis_fm_gdf_reproj.geometry.y
nwis_fm_gdf_reproj['obsprefix'] = nwis_fm_gdf_reproj.index

In [ ]:
gw_dv_gdf_reproj.to_csv('../obs_data/NWIS_FieldMeas_sites.csv', index=False)